In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import time

In [2]:
orders = pd.read_csv('order_brush_order.csv.zip', compression='zip', 
                                 header=0, sep=',', quotechar='"')

In [3]:
orders.shape

(222750, 4)

In [4]:
orders.head(20)

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55
5,31119725718155,173811070,116451780,2019-12-27 12:22:05
6,31151322178251,107921853,166741763,2019-12-27 21:08:43
7,31079856153738,178400128,61272835,2019-12-27 01:17:37
8,31118426867571,147941492,10986763,2019-12-27 12:00:27
9,31133458226149,164933170,135957741,2019-12-27 16:10:59


In [5]:
orders['event_time'] = pd.to_datetime(orders['event_time'])  

In [6]:
orders['shopid'].value_counts()

147941492    11703
61556313      6691
54615708      2640
91799978       707
57189823       687
             ...  
158235164        1
147997211        1
166898182        1
194982850        1
189530112        1
Name: shopid, Length: 18770, dtype: int64

In [7]:
# group_shops = orders.groupby('shopid')

In [2]:
# group_shops.groups

In [ ]:
# group_shops.groups.keys()

## 主要運算的部分

In [19]:
df_result = pd.DataFrame(columns = ["shopid","userid"]) #先建立一個空的DataFrame來裝最後要輸出的結果


##########################################
# 用來計算執行進度的
total_shopid = orders.shopid.nunique() #18770
x = 1
##########################################

for (shopid, group) in orders.groupby('shopid'): # 對 shopid 做 groupby, group 會是每個不同 shopid 各自的 dataframe

    #建立一個空的 DataFrame，要用來存每間店出現在 order brushing period 中的所有資料
    df_order_brushing = pd.DataFrame(columns = ["orderid","shopid","userid","event_time","timestamp"]) 

    # 把時間轉換成timestamp，並根據時間先後做排序
    group['timestamp'] = group["event_time"].values.astype(np.int64) // 10 ** 9 # 轉換成timestamp（全部轉換成多少秒）
    group.sort_values(by=['timestamp'], inplace=True) #根據時間先後做排序
    group = group.reset_index(drop = True)

    # 用類似 window 的方式，把每筆資料加1小時後的區段皆做檢測
    for i in range(group["timestamp"].size): #用 for i in range(group.shape[0]): 也可以
        flag = group.at[i,"timestamp"] # flag 代表當下挑的那筆資料，往後再加3600秒
        mask = (group["timestamp"] >= flag) & (group["timestamp"] <= (flag+3600))
        df_small = group.loc[mask] # df_small 是在這一小時內的資料

        # calculate the concetrate rate
        number_of_orders = df_small.shape[0]
        number_of_unique_buyers = df_small.userid.nunique()
        concentrate_rate = number_of_orders/number_of_unique_buyers

        if (concentrate_rate >= 3): # concentrate_rate >= 3，代表這一段資料裡有 order brushing
            # 把所有在 order brushing period 中的資料都存在 df_order_brushing 這個 DataFrame 裡
            df_order_brushing = pd.concat([df_order_brushing, df_small], axis=0) 

            
    # 計算在 order brusing period 中的每個 userid 的 porportion rate，並存到 df_order_proportion 裡
    total_brushing = df_order_brushing.shape[0] # total number of orders during all order brushing periods
    df_order_proportion = (df_order_brushing["userid"].value_counts()/total_brushing).to_frame(name = "order_proportion")
    df_order_proportion.reset_index(level=0, inplace=True)
    df_order_proportion.columns =['userid', 'order_proportion']

    # find out which user is the suspicious buyer (having higest order proportion)
    max_value = df_order_proportion['order_proportion'].max()
    #use max_order_brushing_list to save the suspicious buyers' userid
    max_order_brushing_list = df_order_proportion[df_order_proportion['order_proportion'] == max_value].userid.tolist()
    max_order_brushing_list.sort() # 讓這些 userid 照數字大小排好（題目要求）

    
    # 把最後要輸出的結果 concat 到 df_result 上面
    if(len(max_order_brushing_list)==0):
        df_result = df_result.append({'shopid': shopid, 'userid':0}, ignore_index=True)
        print("Progress {:2.1%}".format(x / total_shopid), end="\r")
        x = x+1

    elif(len(max_order_brushing_list)==1):
        userid_result = max_order_brushing_list[0]
        df_result = df_result.append({'shopid': shopid, 'userid': userid_result}, ignore_index=True)
        print("Progress {:2.1%}".format(x / total_shopid), end="\r")
        x = x+1
    
    else:
        userid_result = '&'
        userid_result = userid_result.join(map(str, max_order_brushing_list)) #先轉換成 string 後，再用&把每個元素連在一起
        df_result = df_result.append({'shopid': shopid, 'userid': userid_result}, ignore_index=True)
        print("Progress {:2.1%}".format(x / total_shopid), end="\r")
        x = x+1


/Users/leosu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/leosu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


### Save file

In [20]:
import os
print("The file will be saved under this path: ", os.getcwd())
export_csv = df_result.to_csv (r'submission_' + time.strftime("%Y-%m-%d %H%M%S", time.localtime()) + '.csv', 
                             index = None, header = True) 

The file will be saved under this path:  /Users/leosu/Desktop/Shopee_Code_League/competition/01_Order_Brushing


In [21]:
# 測試用，可以挑任一 shopid 的資料出來看
test_group = group_shops.get_group(54257623)
test_group['timestamp'] = test_group["event_time"].values.astype(np.int64) // 10 ** 9
test_group.sort_values(by=['timestamp'], inplace=True)
test_group = test_group.reset_index(drop = True)
test_group